In [1]:
from common import *
import smart_open

In [2]:
with open(join(DATA_FOLDER, 'gold_mongo.json'), 'r') as f:
    gold = json.load(f)

In [ ]:
def iter_docs(fnames):
    for i, fn in enumerate(fnames):
#         logging.info("%s: " % i + fn)
        with GzipFile(fn, 'rb') as f:
            docs = ujson.loads(f.read())
        for k,doc in docs.items():
            if k not in gold.keys():
                yield k,doc

                
class Documents(object):
    def __init__(self, folder, tokens_only=False):
        self.folder = folder
        self.tokens_only = tokens_only
 
    def __iter__(self):
        fnames = glob(join(self.folder, '*.json.gz'))
        for k, doc in iter_docs(fnames):
            unlisted = [w for s in doc for w in s]
            if self.tokens_only:
                yield unlisted
            else:
                # For training data, add tags
                yield gensim.models.doc2vec.TaggedDocument(unlisted, [k])

In [ ]:
folder = join(DATA_FOLDER, 'documents/')

model = Doc2Vec(Documents(folder), size=200, window=8, min_count=5, workers=cpu_count-2, iter=10)

2017-09-29 08:44:02,187 [MainThread  ] [DEBUG]  Fast version of gensim.models.word2vec is being used
2017-09-29 08:44:02,189 [MainThread  ] [INFO ]  collecting all words and their counts
2017-09-29 08:44:02,262 [MainThread  ] [INFO ]  PROGRESS: at example #0, processed 0 words (0/s), 0 word types, 0 tags


In [ ]:
model.save('../data/saved/d2v_model_s200_w8_mc5_iter10')